# 102 category flower classification using ResNet50

## 1. Import Necessary Libraries

In [1]:
import os
from pathlib import Path
import scipy

## 2. Define Data Paths
We set paths to the data directory and metadata files (label and split files).

In [2]:
# Set path to data directory
data_dir = Path("data")

# Set path to label files & split files
label_file = data_dir / "imagelabels.mat"
split_file = data_dir / "setid.mat" 

## 3. Load Dataset Metadata
We'll load the `.mat` files containig the labels and train/test splits using `scipy.io.loadmat()`.

In [3]:
labels_data = scipy.io.loadmat(label_file)
splits_data = scipy.io.loadmat(split_file)

labels_data, splits_data

({'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNX86, Created on: Thu Feb 19 15:43:33 2009',
  '__version__': '1.0',
  '__globals__': [],
  'labels': array([[77, 77, 77, ..., 62, 62, 62]], dtype=uint8)},
 {'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNX86, Created on: Thu Feb 19 17:38:58 2009',
  '__version__': '1.0',
  '__globals__': [],
  'trnid': array([[6765, 6755, 6768, ..., 8026, 8036, 8041]], dtype=uint16),
  'valid': array([[6773, 6767, 6739, ..., 8028, 8008, 8030]], dtype=uint16),
  'tstid': array([[6734, 6735, 6737, ..., 8044, 8045, 8047]], dtype=uint16)})

## 4. Extract label and split indices
We'll extract the 102 labels of the flower and the indices of the images which define which images belong to which split (train/test).

In [4]:
labels = labels_data['labels'].flatten()
train_indices = splits_data['trnid'].flatten()
test_indices = splits_data['tstid'].flatten()
val_indices = splits_data['valid'].flatten()

print(f'First five labels: {labels[:5]}')
print(f'Min value of train_indices: {train_indices.min()}')
print(f'Min value of test_indices: {test_indices.min()}')
print(f'Min value of val_indices: {val_indices.min()}')
print()
print(f'Length of train_indices: {len(train_indices)}')
print(f'Length of test_indices: {len(test_indices)}')
print(f'Length of val_indices: {len(val_indices)}')

First five labels: [77 77 77 77 77]
Min value of train_indices: 28
Min value of test_indices: 1
Min value of val_indices: 17

Length of train_indices: 1020
Length of test_indices: 6149
Length of val_indices: 1020


## 5. Map indices to Image Path

In [5]:
# Load and sort all the image files
data_path = data_dir / "images"
images_files = sorted(list(data_path.glob("*.jpg")))

images_files[:5]

[WindowsPath('data/images/image_00001.jpg'),
 WindowsPath('data/images/image_00002.jpg'),
 WindowsPath('data/images/image_00003.jpg'),
 WindowsPath('data/images/image_00004.jpg'),
 WindowsPath('data/images/image_00005.jpg')]

In [6]:
# Create lists of image paths and corresponding labels for the training set.
# i-1 adjustment is done for matching the indices from matlab to python's zero based system
train_images = [images_files[i-1] for i in train_indices]  
train_labels = [labels[i-1] for i in train_indices]

test_images = [images_files[i-1] for i in test_indices]
test_labels = [labels[i-1] for i in test_indices]


val_images = [images_files[i-1] for i in val_indices]
val_labels = [labels[i-1] for i in val_indices]

# Checking the data
print(f'Min train_images value: {min(train_images)}')
print(f'Min test_values: {min(test_images)}')
print(f'Min val_values: {min(val_images)}')
print()
print(f'Length of train_images: {len(train_images)}')
print(f'Length of test_images {len(test_images)}')
print(f'Length of val_images: {len(val_images)}')

Min train_images value: data\images\image_00028.jpg
Min test_values: data\images\image_00001.jpg
Min val_values: data\images\image_00017.jpg

Length of train_images: 1020
Length of test_images 6149
Length of val_images: 1020


## 6. Define Transformations
We use the default transformations for ResNet50, including resizing, cropping, and normalization.

In [7]:
import torch
from torchvision import transforms
from torchvision import models

pretrained_weights = models.ResNet50_Weights.DEFAULT
common_transforms = pretrained_weights.transforms()

common_transforms

ImageClassification(
    crop_size=[224]
    resize_size=[232]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

## 7. Create custom Dataset Class

In [8]:
from torch.utils.data import Dataset
from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, data_path, labels, transform=None):
        self.data_path = data_path
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data_path)

    def __getitem__(self, idx: int):
        label = self.labels[idx] - 1
        img = transforms.functional.pil_to_tensor(transforms.functional.load_image(data_path))
        if self.transform:
            image = self.transform(img)
        return img, label

train_dataset = CustomDataset(train_images, train_labels, transform=common_transforms)
val_dataset = CustomDataset(val_images, val_labels, transform=common_transforms)
test_dataset = CustomDataset(test_images, test_labels, transform=common_transforms)

print(f'Length of train_dataset: {len(train_dataset)}')
print(f'Length of test_dataset {len(test_dataset)}')
print(f'Length of val_dataset: {len(val_dataset)}')

Length of train_dataset: 1020
Length of test_dataset 6149
Length of val_dataset: 1020


## 8. Create Dataloaders

We create the `train_loader`, `val_loader`, `test_loader` with mini-batch size of 32.
* `train_loader` feeds batches of training data to the model during training.
* `val_loader` is used during training to monitor performance on unseen data and detect overfitting.
* `test_loader` is used after training to evaluate final model performance on a separate test set.
 
The reason of using `shuffle=True` for `train_loader` and `shuffle=False` (default) for `val_loader` and `test_loader`:
* Shuffling is used to break the sequential order of data, helping to improve model generalization and avoid overfitting to data patterns.
* Validation and testing datasets do not require shuffling since they are only used to evaluate the model's performance.

In [23]:
from torch.utils.data import DataLoader

# Define min-batch size
batch_size = 32

# Create the DataLoader for the training dataset
train_loader = DataLoader(train_dataset, 
                          batch_size=batch_size, # Load batch_size samples at a time
                          shuffle=True)    # Randomly shuffle the data at the start of each epoch to improve model generalization

# Create the DataLoader for the validatoin dataset
val_loader = DataLoader(val_dataset, batch_size=batch_size)  # No shuffling needed for validation, as order doesn't affect model evaluation

# Create the DataLoader for the testing dataset
test_loader = DataLoader(test_dataset, batch_size=batch_size) 

## 9. Load Pretrained Model
We follow the following steps to load the pretrained model:
* Load the ResNet50 model with `pretrained_weights`
* Replace the final fully connected layer to output 102 classes
* Freeze the pretrained Layers except the final one

In [13]:
from torch import nn 

# Load the ResNet50 model with pretrained weights 
model = models.resnet50(weights=pretrained_weights)

# Replace the final fully connected layer to output 102 classes
num_classes = 102
model.fc = nn.Linear(model.fc.in_features, num_classes)


In [14]:
# Freeze all layers except the final one to leverage pretrained features
for param in model.parameters():
    param.requires_grad = False
model.fc.requires_grad = True

## 10. Move Model to Device

In [15]:
# Change the device to cuda if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [16]:
# Move the model to current device
model = model.to(device)

## 11. Define Loss and Optimizer

In [18]:
from torchmetrics import Accuracy

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001)
accuracy_fn = Accuracy(task='multiclass', num_classes=102)

## 12. Training Loop

In [19]:
from torch import tqdm
from train import train_model, test_model

epochs = 10
train_loss, train_accuracy = 0, 0

for epoch in tqdm(range(epochs)):
    


    



SyntaxError: incomplete input (2338673895.py, line 13)

In [ ]:
torch.nn.BC